<a href="https://colab.research.google.com/github/Kushagra-1071/Portfolio_Optimization/blob/main/Portfolio_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing 100 companies data.


In [7]:
import yfinance as yf
import pandas as pd

df=pd.read_excel('Ticker_data.xlsx')


# Remove every alternate entry from the DataFrame
new_df = df.iloc[1::2]
new_df['Unnamed: 0'] = new_df['Unnamed: 0'].apply(lambda x: x + '.NS')

# Print the dataframe
print(new_df)


        Unnamed: 0
1         ABFRL.NS
3         AFFLE.NS
5    AARTIDRUGS.NS
7      AARTIIND.NS
9           ACC.NS
..             ...
191  COCHINSHIP.NS
193   CENTRALBK.NS
195  CHAMBLFERT.NS
197    CHOLAFIN.NS
199        CERA.NS

[100 rows x 1 columns]


<ipython-input-7-0adc301fcc89>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Unnamed: 0'] = new_df['Unnamed: 0'].apply(lambda x: x + '.NS')


Getting ticker data from yahoo finance for the data

In [16]:
excel_file='/content/Ticker_data.xlsx'
new_df.to_excel=(excel_file)

LSTM Approach

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


# Load and preprocess the historical asset data
# Put your own stuff here.


# Split the data into training and testing sets
# Put your own stuff here.


# Create and train the LSTM model
model = Sequential()
model.add(LSTM(units=64, input_shape=(num_timesteps, num_features)))
model.add(Dense(units=num_assets))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=10, batch_size=32)


# Compute optimal portfolio allocation
allocation = model.predict(X_test)
# Put your own stuff here.


# Generate rebalanced portfolio weights
rebalanced_weights = rebalance_portfolio(allocation, current_weights)
# Put your own stuff here.

My_Approach

In [19]:
#importing Libraries
import yfinance as yf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from numpy import array
import numpy as np
from scipy.optimize import minimize

In [17]:
#Getting Company data for Nifty 50 stocks
company_tickers = ['ADANIPORTS.NS','ADANIENT.NS','APOLLOHOSP.NS','AXISBANK.NS','ASIANPAINT.NS','BAJAJFINSV.NS','BAJAJ-AUTO.NS','BAJFINANCE.NS','BRITANNIA.NS','BHARTIARTL.NS','BPCL.NS','CIPLA.NS','COALINDIA.NS','DRREDDY.NS','DIVISLAB.NS','EICHERMOT.NS','GRASIM.NS','HINDALCO.NS','HDFCBANK.NS','HDFCLIFE.NS','HCLTECH.NS','HINDUNILVR.NS','HEROMOTOCO.NS','ICICIBANK.NS','ITC.NS','INFY.NS','INDUSINDBK.NS','JSWSTEEL.NS','KOTAKBANK.NS','LT.NS','LTIM.NS','M&M.NS','MARUTI.NS','NESTLEIND.NS','ONGC.NS','NTPC.NS','POWERGRID.NS','RELIANCE.NS','SBIN.NS','SBILIFE.NS','SUNPHARMA.NS','TATASTEEL.NS','TATACONSUM.NS','TATAMOTORS.NS','TCS.NS','TECHM.NS','TITAN.NS','UPL.NS','ULTRACEMCO.NS','WIPRO.NS']
stock_data = yf.download(company_tickers, period='5y')

[*********************100%%**********************]  50 of 50 completed
              Adj Close                                            \
            ADANIENT.NS ADANIPORTS.NS APOLLOHOSP.NS ASIANPAINT.NS   
Date                                                                
2018-11-02   173.520142    323.060760   1137.772705   1205.804932   
2018-11-05   166.504211    319.138733   1148.373413   1202.952393   
2018-11-06   162.502182    315.313538   1111.690063   1197.585815   
2018-11-07   163.786789    315.894592   1115.979614   1209.624512   
2018-11-09   164.330276    326.934418   1132.842163   1252.799316   
...                 ...           ...           ...           ...   
2023-10-27  2261.699951    782.400024   4830.200195   2955.149902   
2023-10-30  2299.100098    785.000000   4841.750000   2967.100098   
2023-10-31  2294.649902    784.750000   4819.899902   2995.699951   
2023-11-01  2217.300049    769.099976   4796.549805   2934.550049   
2023-11-02  2215.300049    774.2

In [22]:
#Train Test Split preparation for LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(stock_data[['Open', 'High', 'Low', 'Close', 'Volume']], stock_data[['Close']], test_size=0.25)

# Normalize the stock data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [23]:
#Building the model
model = Sequential()

# Add an LSTM layer with 128 units
model.add(LSTM(128, input_shape=(X_train.shape[1],)))

# Add a dense layer with 1 unit
model.add(Dense(1))

# Compile the model
model.compile(loss='mse', optimizer='adam')

ValueError: ignored